In [51]:
import numpy as np
import gensim
import string
import networkx as nx

In [52]:
from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils.data_utils import get_file


In [53]:
print('\nPreparing the walks...')
with open('collapsed_node2vec_walks.txt') as infile:
  docs = infile.readlines()
walks = [[node for node in doc.split()] for doc in docs]
print('Num walks:', len(walks))
walklength = len(walks[34])


Preparing the walks...
Num walks: 12820


In [54]:
print(walks)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [57]:
dataset=open("collapsed.edgelist", 'rb')
G=nx.read_edgelist(dataset)
nodelist = list(G.nodes)


In [58]:
print(nodelist)

['57154', '3309', '5879', '23568', '1432', '2004', '8945', '5469', '10014', '7037', '10672', '7046', '4627', '23405', '3329', '2885', '2734', '5966', '5747', '1605', '10252', '868', '648', '11171', '3654', '3315', '3476', '79840', '10458', '55971', '6764', '6461', '6714', '4851', '5664', '2549', '5829', '5295', '5871', '6416', '302', '3622', '4087', '23557', '5603', '7157', '5599', '6464', '5300', '4303', '7531', '678', '3981', '1278', '4209', '2771', '5581', '7186', '5594', '208', '9545', '4086', '58533', '6809', '3643', '5580', '1958', '6647', '6774', '5371', '4790', '5607', '6868', '1147', '9043', '10987', '6886', '2547', '8976', '4690', '5606', '9479', '1956', '6850', '939', '56913', '6050', '283', '1969', '10755', '613', '5716', '8440', '1832', '5663', '10728', '7189', '4615', '4088', '9344', '4691', '3685', '7040', '3091', '55294', '5467', '5970', '4214', '10049', '240', '5578', '6778', '102', '5604', '3688', '7414', '7132', '1386', '306', '51135', '7001', '5914', '960', '7428', 

In [ ]:
embeddings_index = dict()
f = open('C:\\Users\\320052308\\minorproject\\embeddings\\collapsed\\node2vec\\collapsed.embeddings',encoding="utf8")
line1 = f.readline()
vecs = int(line1.split()[0])
dims = int(line1.split()[1])
for line in f:
    values = line.split()
    geneid = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[geneid] = coefs
f.close()
print('Loaded %s node vectors.' % len(embeddings_index))

In [40]:
print(embeddings_index)

{'1': array([-0.014876, -0.229356,  0.023806,  0.067841,  0.09091 , -0.14488 ,
        0.15013 ,  0.075787,  0.046873, -0.02529 , -0.095061,  0.084067,
        0.022556, -0.197516,  0.051331, -0.241755,  0.004764, -0.23258 ,
       -0.031161, -0.013199,  0.233664, -0.01123 ,  0.140365,  0.096059,
        0.068695, -0.165448, -0.100326, -0.012106,  0.011358,  0.17088 ,
       -0.012065,  0.044922, -0.139547, -0.037438, -0.095815, -0.138876,
        0.183298,  0.116599, -0.055275, -0.080746, -0.100899,  0.053135,
        0.141539,  0.179031, -0.131575,  0.127407,  0.09988 , -0.064466,
       -0.029267,  0.157667, -0.028006,  0.174557, -0.022628,  0.148424,
       -0.174256, -0.17907 , -0.059111,  0.088243,  0.086486, -0.033525,
        0.117324, -0.099924, -0.026303,  0.151631, -0.063171, -0.079035,
       -0.073884,  0.059158,  0.136396, -0.156904, -0.089332,  0.054849,
       -0.108355, -0.044886,  0.131026, -0.063695,  0.08619 ,  0.01478 ,
       -0.002258, -0.120929, -0.115947, -0.02

In [41]:
embedding_matrix = np.zeros((vecs, dims))
print(embedding_matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [42]:
print(type(embeddings_index["1"]))
print(type(embedding_matrix[1]))
#embedding_matrix[1]=embeddings_index["1"]
for i in range(len(nodelist)):
    embedding_matrix[i]=embeddings_index[nodelist[i]]
#print(embedding_matrix.shape)
print(embedding_matrix)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[-0.014876   -0.22935601  0.023806   ...  0.074112    0.057659
   0.20126501]
 [-0.028685   -0.21307001  0.047301   ...  0.131276    0.034881
   0.128345  ]
 [-0.013324   -0.230635    0.030984   ...  0.091342    0.055049
   0.181622  ]
 ...
 [-0.02302    -0.198035    0.052784   ...  0.13644201  0.033857
   0.105298  ]
 [-0.035282   -0.20698801  0.058312   ...  0.15886401  0.029401
   0.100663  ]
 [-0.032797   -0.195288    0.069216   ...  0.176257    0.028692
   0.068829  ]]


In [43]:
def node2id(node):
    return nodelist.index(node)
def id2node(idx):
    return nodelist[idx]

In [1]:
train_x = np.zeros([len(walks), walklength], dtype=np.int32)
train_y = np.zeros([len(walks)], dtype=np.int32)
for i in range(len(walks)):
    for j in range(len(walks[i])-1):
        train_x[i][j] = node2id(walks[i][j])
    train_y[i] = node2id(walks[i][-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)

print(train_x[0])
print(walks)
#print(train_y)


NameError: name 'np' is not defined

In [45]:
print(train_x)
print(walks[0])

[[ 5  0  7 ... 12  8  0]
 [ 2  0 14 ... 19 26  0]
 [28 23 25 ... 16 15  0]
 ...
 [19  9 23 ... 15 18  0]
 [ 5 16 10 ... 22 11  0]
 [13 11 12 ... 23 21  0]]
['14', '1', '12', '1', '3', '2', '4', '14', '4', '1', '9', '33', '16', '34', '28', '25', '26', '24', '26', '25', '28', '3', '8', '1', '11', '5', '1', '13', '1', '2', '20', '34', '24', '30', '24', '30', '34', '16', '33', '32', '25', '28', '25', '32', '33', '21', '33', '19', '33', '9', '3', '10', '34', '9', '33', '21', '34', '31', '2', '22', '1', '3', '14', '4', '1', '22', '2', '1', '8', '1', '18', '2', '20', '1', '7', '17', '7', '6', '11', '1']


In [49]:
model = Sequential()
model.add(Embedding(input_dim=vecs, output_dim=dims, weights=[embedding_matrix], trainable=False))
model.add(LSTM(units=dims))
model.add(Dense(units=vecs))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 128)         4352      
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_8 (Dense)              (None, 34)                4386      
_________________________________________________________________
activation_6 (Activation)    (None, 34)                0         
Total params: 140,322
Trainable params: 135,970
Non-trainable params: 4,352
_________________________________________________________________
None


In [50]:
model.fit(train_x, train_y, validation_split=0.4,
          batch_size=128,
          epochs=20,)
loss, accuracy = model.evaluate(train_x, train_y, verbose=0)
print('Accuracy: %f' % (accuracy*100))


Train on 1020 samples, validate on 680 samples
Epoch 1/20
1020/1020 [==============================] - 5s 5ms/step - loss: 3.4431 - acc: 0.0863 - val_loss: 3.3090 - val_acc: 0.0941
Epoch 2/20
1020/1020 [==============================] - 3s 3ms/step - loss: 3.2569 - acc: 0.1353 - val_loss: 3.2342 - val_acc: 0.1324
Epoch 3/20
1020/1020 [==============================] - 3s 3ms/step - loss: 3.1834 - acc: 0.1471 - val_loss: 3.2064 - val_acc: 0.1382
Epoch 4/20
1020/1020 [==============================] - 3s 3ms/step - loss: 3.1571 - acc: 0.1431 - val_loss: 3.1903 - val_acc: 0.1338
Epoch 5/20
1020/1020 [==============================] - 3s 3ms/step - loss: 3.1079 - acc: 0.1520 - val_loss: 3.1262 - val_acc: 0.1426
Epoch 6/20
1020/1020 [==============================] - 3s 3ms/step - loss: 3.0664 - acc: 0.1490 - val_loss: 3.0839 - val_acc: 0.1559
Epoch 7/20
1020/1020 [==============================] - 3s 3ms/step - loss: 3.0302 - acc: 0.1559 - val_loss: 3.0657 - val_acc: 0.1456
Epoch 8/20
1020